<center>
    <h1>NBA Playoff Predictions</h1>
    <h3>Neel Shah</h3>
</center>


**Table of contents**<a id='toc0_'></a>

1. [Introduction](#toc1_)  
   1.1. [Background Information](#toc1_1_)  
   1.2. [Objective](#toc1_2_)  
   1.3. [Configuration](#toc1_3_)
2. [Data Collection](#toc2_)  
   2.1. [Setup](#toc2_1_)  
   2.2. [Data Scraping](#toc2_2_)
3. [Data Processing](#toc3_)
4. [Exploratory Data Analysis & Visualization](#toc4_)
5. [Modeling: Analysis, Hypothesis Testing, & Machine Learning](#toc5_)
6. [Interpretation: Insight & Policy Decision](#toc6_)

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->


# 1. <a id='toc1_'></a>[Introduction](#toc0_)


## 1.1. <a id='toc1_1_'></a>[Background Information](#toc0_)


The [National Basketball Association (NBA)][NBA] is a professional basketball league in the United States. There are [$30$ teams][NBA teams] in the league, divided evenly into $2$ conferences: the Eastern Conference and the Western Conference.

In the regular season, each team plays $82$ games. [NBA regular season standings][NBA standings] are determined by teams' win-loss records within their conferences.

The top $8$ teams from each conference advance to the playoffs. In the event of a tie in the standings, there is a [tie-breaking procedure][NBA tie-breaking procedure] used to determine playoff seeding.

Starting in the $2019\text{-}20$ season, the NBA [added a play-in tournament][Bleacher Report play-in tournament] to give the $9^\text{th}$ and $10^\text{th}$ place teams in each conference the opportunity to earn a spot in the playoffs. It works as follows:

- The $7^\text{th}$ and $8^\text{th}$ place teams play a game to determine the $7^\text{th}$ seed. The winner advances to the playoffs.
- The $9^\text{th}$ and $10^\text{th}$ place teams play an elimination game. The loser is eliminated.
- The loser of the $7/8$ game and the winner of the $9/10$ game play an elimination game to determine the $8^\text{th}$ seed. The winner advances to the playoffs; the loser is eliminated.

Once the final playoff seeding is determined, each team plays an opponent in a best-of-$7$ series. The first to win $4$ games advances to the next round. The first round is followed by the conference semifinals, then the conference finals, then the finals. The team that wins the NBA Finals in the NBA Champion.

The matchups for each round are determined using a traditional bracket structure, shown below:

![NBA Playoff Bracket](nba-playoff-bracket.png "NBA Playoff Bracket")

[NBA]: https://www.nba.com/
[NBA teams]: https://www.nba.com/teams
[NBA standings]: https://www.nba.com/standings
[NBA tie-breaking procedure]: https://ak-static-int.nba.com/wp-content/uploads/sites/2/2017/06/NBA_Tiebreaker_Procedures.pdf
[Bleacher Report play-in tournament]: https://bleacherreport.com/articles/10031906-adam-silver-envisions-nba-play-in-tournament-becoming-a-fixture-in-this-league


## 1.2. <a id='toc1_2_'></a>[Objective](#toc0_)


We want to perform some analysis to see if we can identify factors underlying teams' level of success in the playoffs. Our ultimate goal will be to predict the outcome of the NBA playoffs using data from the regular season.

Can we predict how likely a team is to:

- Make the conference semifinals (i.e. win at least $4$ playoff games)?
- Make the conference finals (i.e. win at least $8$ playoff games)?
- Make the finals (i.e. win at least $12$ playoff games)?
- Win the championship (i.e. win $16$ playoff games)?

These are some of the questions we want to answer as we go through the full data science pipeline.


## 1.3. <a id='toc1_3_'></a>[Configuration](#toc0_)


We'll start by importing the [`Python`][Python] libraries necessary for this project and configuring some things.

[Python]: https://www.python.org/


In [47]:
import warnings
import time
from pathlib import Path
import requests
from bs4 import BeautifulSoup, Comment, MarkupResemblesLocatorWarning
import pandas as pd

warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning, module="bs4")

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}


# 2. <a id='toc2_'></a>[Data Collection](#toc0_)


## 2.1. <a id='toc2_1_'></a>[Setup](#toc0_)


Now, we need to collect data that we can use in our analysis. For this, we will scrape data from [Basketball Reference][Basketball Reference], a site that provides historical basketball data.

We will use data from the $2002\text{-}03$ season ([when the NBA switched to the current playoff format, where every series is best-of-$7$][NBA playoffs history]) to the $2022\text{-}23$ season (the current season). For each season, we will scrape the following information:

- Per Game Stats from Season Summary page
- Advanced Stats from Season Summary page
- Expanded Standings from Standings page
- Advanced Stats from Playoffs Summary page

For convenience, we will define a function called `pages_to_scrape`. Given a season, it will return a dictionary which maps the URL of the page we will be scraping to the list of information about each table we will be scraping from that page.

Each element in the list will be in the form of a dictionary with $2$ items:

- The `id` of the `HTML` `table` element we will be scraping from the page
- The `path` where we will be storing the table as a `CSV`

[Basketball Reference]: https://www.basketball-reference.com/
[NBA playoffs history]: https://www.nba.com/magic/history-no-1-vs-no-8-nba-playoffs-20200810


In [48]:
def pages_to_scrape(season):
    return {
        f"https://www.basketball-reference.com/leagues/NBA_{season}.html": [
            {
                "id": "per_game-team",
                "path": f"data/raw/{season}/regular_season/per_game_stats.csv",
            },
            {
                "id": "advanced-team",
                "path": f"data/raw/{season}/regular_season/advanced_stats.csv",
            },
        ],
        f"https://www.basketball-reference.com/leagues/NBA_{season}_standings.html": [
            {
                "id": "expanded_standings",
                "path": f"data/raw/{season}/regular_season/standings.csv",
            }
        ],
        f"https://www.basketball-reference.com/playoffs/NBA_{season}.html": [
            {
                "id": "advanced-team",
                "path": f"data/raw/{season}/playoffs/advanced_stats.csv",
            }
        ],
    }


## 2.2. <a id='toc2_2_'></a>[Data Scraping](#toc0_)


To scrape the data from each page, we will do the following:

- Perform an `HTTP` `GET` request to the appropriate URL using the [`Requests`][Requests] library.
- Parse the webpage using [`Beautiful Soup`][Beautiful Soup].
- Find the `HTML` `table` with the appropriate `id`.
- Read the parsed `HTML` `table` into a `DataFrame` using [`pandas`][pandas].
- Ensure that the appropriate `path` exists in the filesystem using [`pathlib`][pathlib].
- Write the `DataFrame` to a `CSV` file at the appropriate `path` using [`pandas`][pandas].

In our approach, there are a few issues that we have to address as well:

- To save time, if the `CSV` files for a page already exist, we will not re-scrape the page.
- It appears that some of the `table` elements are hidden inside of `HTML` comments, so we have to look there if a `table` can't be found normally.
- To avoid hitting rate limits (from making too many requests in a given time period), we have to add a $10$ second `sleep` between each `HTTP` `GET` request using the [`time`][time] library.

[Requests]: https://requests.readthedocs.io/en/latest/
[Beautiful Soup]: https://beautiful-soup-4.readthedocs.io/en/latest/
[pandas]: https://pandas.pydata.org/
[pathlib]: https://docs.python.org/3/library/pathlib.html
[time]: https://docs.python.org/3/library/time.html


In [49]:
for season in range(2003, 2023 + 1):

    pages = pages_to_scrape(season)

    for url, infoList in pages.items():

        if all([Path(info["path"]).exists() for info in infoList]):
            continue

        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.content, "html.parser")

        for info in infoList:

            if Path(info["path"]).exists():
                continue

            table = soup.find("table", id=info["id"])
            if table is None:
                for comment in soup.find_all(
                    string=lambda text: isinstance(text, Comment)
                ):
                    comment_soup = BeautifulSoup(comment, "html.parser")
                    table = comment_soup.find("table", id=info["id"])
                    if table is not None:
                        break

            df = pd.read_html(str(table))[0]
            Path(info["path"]).parent.mkdir(parents=True, exist_ok=True)
            df.to_csv(info["path"], index=False)

        time.sleep(10)


# 3. <a id='toc3_'></a>[Data Processing](#toc0_)


# 4. <a id='toc4_'></a>[Exploratory Data Analysis & Visualization](#toc0_)


# 5. <a id='toc5_'></a>[Modeling: Analysis, Hypothesis Testing, & Machine Learning](#toc0_)


# 6. <a id='toc6_'></a>[Interpretation: Insight & Policy Decision](#toc0_)
